#### Connect to MySQL database. 
We need
- Start the DB with docker
- Connector/Driver -> pyMySQL
- Toolkit -> SQLAlchemy

In [1]:
!pip install pymysql
!pip install sqlalchemy

In [2]:
import sqlalchemy
import pandas as pd

In [3]:
# specify database configurations
config = {
    'host': 'localhost',
    'port': 3306,
    'user': 'root',
    'password': 'my_password',
    'database': 'my_db'
}
db_user = config.get('user')
db_pwd = config.get('password')
db_host = config.get('host')
db_port = config.get('port')
db_name = config.get('database')

# connection string
connection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'

# connect to database
engine = sqlalchemy.create_engine(connection_str)
connection = engine.connect()
print(f"Connected: {not connection.closed}")
# with engine.connect() as connection:
#     # do things...


Connected: True


#### Import csv files

In [4]:
# Read from tab separated value file
df_customers = pd.read_csv("customers.tsv", sep="\t")
df_customers.head()

,customerid,firstname,lastname,city,state
0,10101,John,Gray,Lynden,Washington
1,10298,Leroy,Brown,Pinetop,Arizona
2,10299,Elroy,Keller,Snoqualmie,Washington
3,10315,Lisa,Jones,Oshkosh,Wisconsin
4,10325,Ginger,Schultz,Pocatello,Idaho


In [5]:
# Read from tab separated value file
df_products = pd.read_csv("products.tsv", sep="\t")
df_products.head()

,customerid,order_date,item,quantity,price
0,10330,30-Jun-1999,Pogo stick,1,28.0
1,10101,30-Jun-1999,Raft,1,58.0
2,10298,01-Jul-1999,Skateboard,1,33.0
3,10101,01-Jul-1999,Life Vest,4,125.0
4,10299,06-Jul-1999,Parachute,1,1250.0


#### Write tables to database
SQLAlchemy will define some extension methods like `to_sql` that integrates to Pandas and add DB features

In [6]:
affected_rows = df_customers.to_sql('customers', engine, if_exists='replace', index=False)
print(f"Affected rows: {affected_rows}")

Affected rows: 17


In [7]:
df_products.to_sql('products', engine, if_exists='replace', index=False)
print(f"Affected rows: {affected_rows}")

Affected rows: 17


#### Additional parameters

**if_exists** – This parameter is used to decide what should be done in case the table already exists in the database. By default, pandas will not be able to write data into this table and will eventually throw an error. You can customize it by providing a value of “REPLACE” if you would like to drop and create a new table every time the code is executed. Also, you can pass a value of “APPEND” if you want to add new records into the table on each execution

**schema** – By default, pandas will write data into the default schema for the database. In PostgreSQL, it is the “public” schema, whereas, in SQL Server, it is the “dbo” schema. If you want it to create a table in a different schema, you can add the name of the schema as value to this parameter

**index** – This is a Boolean field which adds an INDEX column to the table to uniquely identify each row when the value is set to TRUE

**chunksize** – This can be referred to as a batch of data being inserted to the table instead of one row at a time. You can specify an integer value, and that will be the size of the batch that will be used to insert the data. This feature is useful if you have a really large dataset, and you want to bulk insert data

**dtype** – This is a dictionary that accepts the column names and their datatypes if we need to explicitly declare the datatypes of the fields that are in the dataframe. The key in the dictionary is the name of the column, and the value is the datatype. This is recommended if you want greater control over declaring the datatypes of your table and do not want to rely upon the module to do it for you

#### interact with database

In [8]:
from sqlalchemy import text


query = text(
"""
    ALTER TABLE `customers` 
    ADD PRIMARY KEY (`customerid`);
""")
connection.execute(query)

In [9]:
from sqlalchemy import text


query = text(
"""
   ALTER TABLE `products` 
   ADD FOREIGN KEY (`customerid`) REFERENCES customers(`customerid`);
""")
connection.execute(query)

#### read from database

In [10]:
df_customers = pd.read_sql('customers', engine)
df_products = pd.read_sql('products', engine)

In [11]:
df_customers.head()

,customerid,firstname,lastname,city,state
0,10101,John,Gray,Lynden,Washington
1,10298,Leroy,Brown,Pinetop,Arizona
2,10299,Elroy,Keller,Snoqualmie,Washington
3,10315,Lisa,Jones,Oshkosh,Wisconsin
4,10325,Ginger,Schultz,Pocatello,Idaho


In [12]:
df_products.head()

,customerid,order_date,item,quantity,price
0,10330,30-Jun-1999,Pogo stick,1,28.0
1,10101,30-Jun-1999,Raft,1,58.0
2,10298,01-Jul-1999,Skateboard,1,33.0
3,10101,01-Jul-1999,Life Vest,4,125.0
4,10299,06-Jul-1999,Parachute,1,1250.0
